How do PM2.5 concentration affect asthma rates by country

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [73]:
df = pd.read_csv('./asthma-aqi/data.csv')

# Refining the dataframe
df_filtered = df[df['Dim1'] == 'Total']
df_filtered = df_filtered[['Location', 'Period', 'Value']]

df_filtered['Value'] = df_filtered['Value'].str.split().str[0].astype(float) # we do not care about the confidence interval, only the val

# sort location column to keep it alphabetical
df_filtered = df_filtered.sort_values(by='Location').reset_index(drop=True)

# pivot dataframe to add years as columns and concentration (pm 2.5) as the row values
new_df = df_filtered.pivot(index='Location', columns='Period', values='Value')

new_df

Period,2010,2011,2012,2013,2014,2015,2016,2017,2018
Location,,,,,,,,,
Afghanistan,68.97,66.94,68.26,72.18,68.06,67.20,64.00,61.78,70.25
Albania,21.95,23.00,21.70,19.77,19.13,18.98,18.15,19.01,19.76
Algeria,21.27,21.56,22.76,21.19,22.64,22.49,23.06,23.33,22.73
Andorra,11.43,11.66,10.80,10.15,9.35,9.94,9.06,9.31,9.21
Angola,24.20,24.67,24.01,24.53,24.12,25.03,26.08,25.44,25.85
...,...,...,...,...,...,...,...,...,...
Viet Nam,22.37,20.67,21.67,22.62,20.08,20.47,22.37,20.10,20.09
Yemen,38.83,41.03,42.49,41.02,38.27,41.06,39.91,41.32,41.62
Zambia,17.09,17.33,17.55,17.24,17.29,17.43,17.32,16.99,16.99


In [74]:
df_1 = pd.read_excel('./asthma-aqi/asthma.xlsx')

df_1.rename(columns={'Country/Area': 'Location'}, inplace=True)
df_1.columns = df_1.columns.astype(str) # To ensure we can drop columns
df_1.drop(columns=['2019'], inplace=True) # To keep data comparable, we dropped 2019 values as our AQI dataset did not have data from that year.
df_1.set_index('Location', inplace=True)

df_1


,2010,2011,2012,2013,2014,2015,2016,2017,2018
Location,,,,,,,,,
Afghanistan,0.059,0.057,0.055,0.053,0.051,0.050,0.049,0.048,0.048
Albania,0.031,0.031,0.030,0.029,0.028,0.027,0.027,0.027,0.027
Algeria,0.033,0.033,0.032,0.031,0.031,0.030,0.030,0.030,0.030
American Samoa,0.023,0.022,0.021,0.019,0.018,0.017,0.017,0.016,0.016
Andorra,0.070,0.069,0.068,0.068,0.067,0.067,0.066,0.066,0.066
...,...,...,...,...,...,...,...,...,...
South-East Asia (WHO),0.028,0.028,0.027,0.026,0.025,0.024,0.024,0.024,0.023
Sub-Saharan Africa (WB),0.040,0.040,0.039,0.038,0.037,0.036,0.036,0.035,0.035
Upper-middle-income countries,0.029,0.029,0.029,0.028,0.028,0.028,0.027,0.027,0.027


**Before filtering both dataframes to include common countries, we will check countries that exist in both dataframes and rename a certain dataframe to match country names. (For example: One dataset has United States while the other has United States of America).**

In [79]:
# Renaming countries from aqi dataframe to ones from the asthma dataframe
# Differences in names were calculated manually
countries_dict = {
    'Papua New Guinea': 'Guinea',
    'Syrian Arab Republic': 'Syria',
    'Congo': 'Democratic Republic of Congo',
    'Russian Federation': 'Russia',
    'Brunei Darussalam': 'Brunei',
    'Guinea': 'Guinea-Bissau',
    'United Republic of Tanzania': 'Tanzania',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Nigeria': 'Niger',
    'Republic of Moldova': 'Moldova',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Iran (Islamic Republic of)': 'Iran',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Sudan': 'South Sudan',
    'United States of America': 'United States',
    'Democratic Republic of the Congo': 'Congo',
    'Samoa': 'American Samoa',
    'Netherlands (Kingdom of the)': 'Netherlands'
}

new_df.rename(index=countries_dict, inplace=True)

**We will create our final dataframe (one that will be used for analysis/visualization purposes) based on countries that exist in both of the above dataframes. Countries that are not within the intersection of the two dataframes will not be accounted for in the analysis.**

In [80]:
countries = new_df.index.intersection(df_1.index)

# Filtering both dataframes to only include common countries/areas from both datasources
final_df_aqi = new_df.loc[countries]
final_df_asthma = df_1.loc[countries]

**We will now plot a line plot showing the asthma rates of the top 5 countries based on GDP and the bottom 5 countries based on GDP. We gathered the data for this from an external source (https://ourworldindata.org/grapher/national-gdp-fouquin-current-gbp?tab=table&time=latest)**